<a href="https://colab.research.google.com/github/NavanjanaLAV/SE4050-deeplearning-2025/blob/IT22609908-bert-distil/bert_distil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
pip install torch transformers datasets pandas scikit-learn

In [4]:
# Load dataset
url = "https://raw.githubusercontent.com/NavanjanaLAV/SE4050-deeplearning-2025/main/Dataset/fake_or_real_news_cleaned.csv"
df = pd.read_csv(url)

In [8]:
#Check first few rows
print(df.head())

                                               title  label
0  ben stein calls th circuit court committed cou...      0
1  trump drops steve bannon national security cou...      1
2  puerto rico expects us lift jones act shipping...      1
3  oops trump accidentally confirmed leaked israe...      0
4     donald trump heads scotland reopen golf resort      1


## Step 1 -  Split into Train and Test

In [7]:
from sklearn.model_selection import train_test_split

Basic Data Checks

In [9]:
# Check for missing values
print("Missing values:")
print(df.isnull().sum())

Missing values:
title    0
label    0
dtype: int64


In [10]:
# Drop rows where 'title' or 'label' is missing
df = df.dropna(subset=['title', 'label'])

In [11]:
# Make sure label is int (0/1)
df['label'] = df['label'].astype(int)

In [12]:
# Check class distribution
print("\nLabel distribution:")
print(df['label'].value_counts())


Label distribution:
label
0    23481
1    21417
Name: count, dtype: int64


Split into Train & Validation Sets

In [13]:
# Extract texts and labels
texts = df['title'].tolist()      # list of headlines
labels = df['label'].tolist()     # list of 0s and 1s

In [14]:
# Split: 80% train, 20% validation
train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts,
    labels,
    test_size=0.2,           # 20% for validation
    stratify=labels,         # keep same ratio of 0/1 in both sets
    random_state=42          # for reproducibility
)

In [15]:
# Print sizes
print(f"Training samples: {len(train_texts)}")
print(f"Validation samples: {len(val_texts)}")

Training samples: 35918
Validation samples: 8980


Save Splits to Disk

In [16]:
# Create DataFrames for each split
train_df = pd.DataFrame({'title': train_texts, 'label': train_labels})
val_df = pd.DataFrame({'title': val_texts, 'label': val_labels})

In [17]:
# Save to CSV
train_df.to_csv('train.csv', index=False)
val_df.to_csv('val.csv', index=False)

In [18]:
# Verify splits are stratified (balanced)
print("Train label distribution:")
print(pd.Series(train_labels).value_counts())

print("\nValidation label distribution:")
print(pd.Series(val_labels).value_counts())

Train label distribution:
0    18785
1    17133
Name: count, dtype: int64

Validation label distribution:
0    4696
1    4284
Name: count, dtype: int64


## Step 2 - Tokenize Text with BERT Tokenizer